# Two-Sample and ANOVA Tests

# 1. Performing t-tests

<b>1.1 Hypothesis testing workflow</b>

You've seen the hypothesis testing workflow for the one sample case where you compared a sample mean to a hypothesized value, and the two sample case where you compared two sample means. In both cases, the workflow shares common steps.

Place the hypothesis testing workflow steps in order from first to last.

![Alt text](Hypothesis%20testing%20workflow.png)

Regardless of the type of hypothesis test you are performing, it will have a workflow that follows this format.

<b>1.2 Two sample mean test statistic</b>

The hypothesis test for determining if there is a difference between the means of two populations uses a different type of test statistic to the z-scores you saw in Chapter 1. It's called "t", and it can be calculated from three values from each sample using this equation.

 
$$
  t = \frac{\bar{x}_{child}-\bar{x}_{adult}}{\sqrt{\frac{s_{child}^2}{n_{child}}+\frac{s_{adult}^2}{n_{adult}}}}
$$
 

While trying to determine why some shipments are late, you may wonder if the weight of the shipments that were on time is less than the weight of the shipments that were late. The late_shipments dataset has been split into a "yes" group, where late == "Yes" and a "no" group where late == "No". The weight of the shipment is given in the weight_kilograms variable.

The sample means for the two groups are available as xbar_no and xbar_yes. The sample standard deviations are s_no and s_yes. The sample sizes are n_no and n_yes. 

In [17]:
# Import the late_shipment dataframe
import pandas as pd
import numpy as np

late_shipments = pd.read_feather("C:\\Users\\yazan\\Desktop\\Data_Analytics\\9-Introduction to Hypothesis Testing\\Datasets\\late_shipments.feather")
xbar = late_shipments.groupby('late_delivery')['weight_kilograms'].mean()
s = late_shipments.groupby('late_delivery')['weight_kilograms'].std()
n = late_shipments.groupby('late_delivery')['weight_kilograms'].count()

xbar_no = xbar[0]
xbar_yes = xbar[1]

s_no = s[0]
s_yes = s[1]

n_no = n[0]
n_yes = n[1]

# Calculate the numerator of the test statistic
numerator = xbar_no - xbar_yes

# Calculate the denominator of the test statistic
denominator = np.sqrt((s_no**2/n_no) + (s_yes**2/n_yes))

# Calculate the test statistic
t_stat = numerator/denominator

# Print the test statistic
print(t_stat)

-2.3936661778766433


When testing for differences between means, the test statistic is called 't' rather than 'z', and can be calculated using six numbers from the samples. Here, the value is about -2.39 or 2.39, depending on the order you calculated the numerator.